In [5]:
import numpy as np

In [1]:
import importcsv

print('Importing csv file...........')
small_dataset = importcsv.opencsv('processedspeechfinal.csv')  

Importing csv file...........


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=0.001)
tfidf_matrix = tfidf_vectorizer.fit_transform(small_dataset['processed_speeches'])

num of comp = 30 => 30 groups of speeches?

No, `n_components` is the number of concepts we want to reduce our matrix to.

In [3]:
from sklearn.decomposition import TruncatedSVD

# it takes about 3 minutes for 500 components
svd = TruncatedSVD(n_components=500) 

U = svd.fit_transform(tfidf_matrix)
S = svd.singular_values_
V = svd.components_

In [8]:
import pickle

def saveUSV(U, S, V, filename):
    pack = [U, S, V]
    with open(filename, 'wb') as fo:
        pickle.dump(pack, fo)

def loadUSV(filename):
    with open(filename, 'rb') as fo:
        r = pickle.load(fo)
        U, S, V = r[0], r[1], r[2]
        return U,S,V
filename = 'USV.pkl'
# saveUSV(U,S,V, filename) 
# U,S,V = loadUSV(filename)

In [ ]:
print(V)
print('S', S)

In [26]:
# Αυτός ο κώδικας τυπώνει τις λέξεις-κλειδιά ανά θεματική περιοχή που εντοπίστηκε
# Δηλαδή για το concept 0 οι λέξεις κλειδιά που το περιγράφουν είναι οι 
# κυρι, υπουργ, υπαρχ, κυβερνης, πολιτικ ....
feature_names = tfidf_vectorizer.get_feature_names_out()
top_keywords_per_concept = 10
top_keywords_dict = {}
for i in range(len(S)): # for all n_components
    top_indices = np.argsort(V[i])[::-1][:top_keywords_per_concept]

    top_keywords = [feature_names[index] for index in top_indices]
    top_keywords_dict[i] = top_keywords
    # print(f'Top keywords for concept i: {i}: {top_keywords}')
top_keywords_dict

{0: ['κυρι',
  'υπουργ',
  'υπαρχ',
  'κυβερνης',
  'πολιτικ',
  'γιατι',
  'χωρ',
  'συναδελφ',
  'αρθρ',
  'νομοσχεδι'],
 1: ['αρθρ',
  'τροπολογι',
  'υπουργ',
  'προεδρ',
  'παραγραφ',
  'κυρι',
  'νομοσχεδι',
  'διαταξ',
  'βουλ',
  'δεκτ'],
 2: ['αρθρ',
  'παραγραφ',
  'δημοσι',
  'υγει',
  'υπηρεσι',
  'διαταξ',
  'υπουργει',
  'νοσοκομει',
  'ρυθμις',
  'αφορ'],
 3: ['υγει',
  'νοσοκομει',
  'υπουργ',
  'γιατρ',
  'υπουργει',
  'εργ',
  'ερωτης',
  'υπαρχ',
  'περιοχ',
  'κεντρ'],
 4: ['τροπολογι',
  'ευρω',
  'αγροτ',
  'αγροτικ',
  'δεκτ',
  'αριθμ',
  'τραπεζ',
  'δισεκατομμυρι',
  'υπουργ',
  'παραγωγ'],
 5: ['τροπολογι',
  'υγει',
  'νοσοκομει',
  'κοινωνικ',
  'πολιτικ',
  'νομοσχεδι',
  'γιατρ',
  'εκπαιδευτικ',
  'εκπαιδευς',
  'αριθμ'],
 6: ['αρθρ',
  'υγει',
  'νοσοκομει',
  'εργαζομεν',
  'ευρω',
  'ασφαλιστικ',
  'γιατρ',
  'παραγραφ',
  'λε',
  'γιατι'],
 7: ['κυρι',
  'αρθρ',
  'εκπαιδευτικ',
  'συναδελφ',
  'βουλ',
  'πρου',
  'μαθητ',
  'σχολει',
  'συνεδριας',


In [25]:
# Let's try to find the most important keywords for each document
# Ο Παπαδοπουλος μηδενίζει τα r-k σ του διαγώνιου πίνακα S και παίρνει μόνο τα k σημαντικότερα
# Εδώ δεν μηδενίζω τίποτα, απλά τυπώνω τα 10 πρώτα σημαντικότερα keywords
# Μάλλον θα πρέπει να κάνουμε κι έναν καθαρισμό στις λέξεις που έχουμε, να μην κρατάμε όσες έχουν πχ ψηφία μέσα?
top_concepts_per_document = 10
# np.argsort does the following: say x = np.array([3, 1, 2])
# then np.argsort(x) would produce
# array([1, 2, 0])
top_concepts_dict = {}
for i in range(len(U)):
    top_indices = np.argsort(U[i])[::-1][:top_concepts_per_document]
    
    top_concepts = [index for index in top_indices]
    top_concepts_dict[i] = top_concepts
    # print(f'Top concepts per document {i}: {top_concepts}')
top_concepts_dict

{0: [175, 0, 27, 178, 122, 10, 1, 158, 7, 173],
 1: [0, 1, 50, 115, 458, 73, 121, 457, 15, 7],
 2: [0, 105, 109, 50, 44, 1, 110, 65, 15, 122],
 3: [0, 12, 364, 41, 1, 16, 73, 149, 15, 94],
 4: [177, 0, 65, 180, 41, 174, 24, 246, 165, 156],
 5: [288, 278, 0, 302, 268, 270, 1, 48, 27, 294],
 6: [0, 195, 214, 273, 176, 163, 237, 7, 197, 43],
 7: [37, 0, 27, 224, 215, 218, 71, 230, 60, 89],
 8: [0, 5, 27, 37, 252, 199, 6, 210, 395, 12],
 9: [0, 194, 242, 238, 50, 36, 477, 479, 175, 240],
 10: [0, 435, 396, 431, 104, 366, 34, 44, 207, 231],
 11: [491, 469, 387, 395, 0, 484, 400, 21, 467, 385],
 12: [0, 229, 43, 57, 16, 5, 168, 76, 209, 75],
 13: [0, 5, 2, 28, 11, 16, 52, 59, 95, 96],
 14: [216, 95, 0, 115, 130, 78, 125, 126, 16, 119],
 15: [216, 0, 447, 457, 470, 114, 83, 423, 204, 464],
 16: [31, 0, 246, 32, 245, 200, 244, 106, 55, 85],
 17: [0, 16, 12, 97, 303, 8, 435, 5, 21, 14],
 18: [0, 16, 247, 333, 306, 168, 296, 131, 14, 21],
 19: [0, 16, 21, 459, 22, 86, 101, 95, 159, 454],
 20: [0

Το παρακάτω κελί δημιουργεί ένα dictionary, όπου κάθε key αντιστοιχεί σε ένα speech και κάθε value αποτελείται από δέκα 10άδες keywords, οι οποίες περιγράφουν το κάθε ένα από τα top10 concepts που εκφράζουν την ομιλία.

In [31]:
# Let us now connect the concepts each document is related to, to the keywords each concept is described by
from collections import defaultdict

# This following dictionary is where all the keywords that describe each speech are stored in
per_speech_keywords = defaultdict(list)
for docID, concepts in top_concepts_dict.items():
    # print(f'Speech {docID} is described by ', end='')
    for concept in concepts:
        # print(top_keywords_dict[concept])
        per_speech_keywords[docID].append(top_keywords_dict[concept])


In [40]:
per_speech_keywords[0][0]

['προεδρικ',
 'διαταγμ',
 'διαταγματ',
 'δικαιωματ',
 'κρατ',
 'δικαιωμ',
 'λεξ',
 'δυνατοτητ',
 'υποθες',
 'δικαιοσυν']

In [37]:
import json 
import pickle

with open('per_speech_keywords.pkl', 'wb') as fp:
    # json.dump(per_speech_keywords, fp)
    pickle.dump(per_speech_keywords, fp)

In [4]:
# V30 = V[:30,:]

# keep only the first 50 values of S, the rest must be 0
S50 = [v if idx < 50 else 0 for idx, v in enumerate(S)] 

In [5]:
import numpy as np

# M = np.dot(tfidf_matrix, V30.transpose())
A = np.dot(U, S50)

## Let's try to keep the most important concepts
We'll keep 80% of the concepts

In [7]:
total_energy = sum(e*e for e in S)
# we want 80% of the initial energy
goal = 0.8 * total_energy

# Then keep summing elements of S until you've reached the goal energy! Simple
# This way, the number of concepts we'll keep is automatically calculated

Sk = np.zeros_like(S) # make an empty array of size like S
energy = 0 # keep track of energy, until we've reached the goal
for i, e in enumerate(S): # could this be done with list comprehension? 
    # irrelevant, wouldn't really save much time, for 500-1000 iterations anyway
    Sk[i] = e
    energy += e ** 2
    if energy > goal:
        print(f'Stopping at iteration {i} and value of e: {e}')
        break

Sk

Stopping at iteration 554 and value of e: 10.30249588655432


array([124.7694159 ,  59.70213408,  51.88242396,  49.90511528,
        41.1485064 ,  39.69715821,  35.534353  ,  34.70651034,
        33.66151611,  33.10387845,  32.0052946 ,  31.32628592,
        30.61204948,  29.73994903,  29.72896001,  29.04100894,
        28.5905262 ,  27.94408407,  27.17744626,  26.66462102,
        26.20927103,  25.81972183,  25.4213688 ,  25.35915305,
        24.81575383,  24.6565024 ,  24.25747936,  24.00568821,
        23.74504941,  23.56189278,  23.26611173,  22.95700771,
        22.63591051,  22.56541173,  22.47630253,  22.33053994,
        22.18612475,  21.92920782,  21.8351794 ,  21.67188581,
        21.4675602 ,  21.32083771,  21.28097702,  21.1810047 ,
        20.98661726,  20.79511465,  20.70289174,  20.66110451,
        20.61345628,  20.48600971,  20.34530136,  20.26252496,
        20.16940451,  20.08172666,  19.96694455,  19.87097604,
        19.73302301,  19.70129067,  19.6805772 ,  19.54706066,
        19.30497407,  19.27592715,  19.26851858,  19.11